In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from collections import Counter
from sklearn.model_selection import train_test_split

from tensorflow.keras.datasets import imdb
from tensorflow.keras.datasets import reuters
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Flatten, Embedding, Dropout, Conv1D, MaxPooling1D
from tensorflow.keras.utils import pad_sequences, to_categorical

In [12]:
!pip install JPype1

In [2]:
kedi = pd.read_csv("kedi.csv", encoding='cp949')

In [3]:
np.unique(kedi['소분류'])

array(['OA', '간호', '건강·미용', '건설·건축·토목', '게임', '결혼·가족·가정', '경영이론', '경영정보',
       '경제', '과학이론·교양', '과학일반', '관광·호텔', '교육학', '교통·운송', '국사', '국악·민요',
       '그림·회화', '글씨디자인', '금융재무', '기계·금속', '기타외국어', '꽃예술', '논술·구술',
       '논술·글쓰기', '농업·원예', '다도', '대중음악', '데이터베이스', '독서', '동·서양문학',
       '디자인이론·교양', '레저·스포츠', '레크리에이션', '마술', '마케팅', '멀티미디어', '무용이론·교양',
       '문학이론·문예창작', '문화', '미술', '미술이론·교양', '법학', '보건', '부동산', '사진·만화',
       '사회복지', '사회일반', '사회학', '생물·화학·물리·천문', '생산관리', '생활공예', '생활무용',
       '생활요리', '서양악기', '서예·서화', '성악', '소방·경찰·경호', '수학', '수화(손언어)', '식생활',
       '신문·방송·출판', '실내인테리어', '심리·상담', '아동발달', '아동지도', '언어', '여성', '역사',
       '연극', '영상예술', '영어', '운영체제', '윤리', '음악', '음악이론·교양', '응용치료', '의료',
       '의상디자인', '인문교양', '인터넷', '일러스트레이션', '일반 기타', '일본어', '입시 기타',
       '재료·정밀', '전기·전자', '전략·국제경영', '전통공예', '전통무용', '전통악기', '정치', '정치학',
       '제2외국어', '제과제빵', '조직관리', '조형', '종교', '종교음악', '중국어', '진로교육', '철학',
       '체육이론·교양', '취업·창업', '컴퓨터기초', '통계', '통신', '풍수·사주', '프로그래밍', '한국문학',
    

In [4]:
import konlpy

In [15]:
from konlpy.tag import Kkma, Komoran, Okt, Twitter

In [20]:
# pos는 품사를 이야기 해 준다.
okt = Okt()
kkma = Kkma()

In [18]:
sentence = "아버지가방에들어가신다"

In [26]:
print(okt.pos(sentence))
print(kkma.pos(sentence))

[('아버지', 'Noun'), ('가방', 'Noun'), ('에', 'Josa'), ('들어가신다', 'Verb')]
[('아버지', 'NNG'), ('가방', 'NNG'), ('에', 'JKM'), ('들어가', 'VV'), ('시', 'EPH'), ('ㄴ다', 'EFN')]


In [25]:
kkma.nouns(sentence), okt.nouns(sentence)

(['아버지', '아버지가방', '가방'], ['아버지', '가방'])

In [60]:
data = pd.read_csv("kedi.csv",encoding="cp949")
X = data['프로그램명']
y = data['소분류']

In [61]:
# 명사만 가져오기
X = X.apply(okt.nouns)

In [62]:
# 가져온 명사들을 리스트에 담기
words = []
for word in X:
    words.extend(word)

In [63]:
# 고유한 값만 뽑기
words_list = list(set(words))

In [64]:
word_to_index = {v:k for k, v in enumerate(words_list, 1)}

In [65]:
index_to_word = {v:k for k, v in word_to_index.items()}

In [68]:
def make_index(x):
    for idx, word in enumerate(x):
        x[idx] = word_to_index.get(word)
    return x

In [69]:
X = X.apply(make_index)

In [72]:
X = X[1:]
y = y[1:]

In [82]:
index_to_subject = dict(zip(np.arange(np.unique(y.values).size),np.unique(y.values)))

In [83]:
subject_to_index = {v:k for k, v in index_to_subject.items()}

In [86]:
def make_subject_index(x):
    x = subject_to_index.get(x)
    return x

In [90]:
y = y.apply(make_subject_index)

In [92]:
y = to_categorical(y)

In [104]:
X = pad_sequences(X, maxlen=21)

In [107]:
model = Sequential(
[
    Embedding(16706,8,input_length=21),
    Conv1D(10,2,activation='relu'),
    MaxPooling1D(2),
    Flatten(),
    Dense(256,activation='relu'),
    BatchNormalization(),
    Dense(117,activation='softmax')
])

In [108]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')

In [109]:
model.fit(X,y,epochs=10)

Epoch 1/10


3912/3912 [==============================] - 11s 2ms/step - loss: 1.7441 - accuracy: 0.5989
Epoch 2/10
3912/3912 [==============================] - 9s 2ms/step - loss: 1.0287 - accuracy: 0.7483
Epoch 3/10
3912/3912 [==============================] - 9s 2ms/step - loss: 0.8610 - accuracy: 0.7840
Epoch 4/10
3912/3912 [==============================] - 9s 2ms/step - loss: 0.7805 - accuracy: 0.7999
Epoch 5/10
3912/3912 [==============================] - 9s 2ms/step - loss: 0.7241 - accuracy: 0.8117
Epoch 6/10
3912/3912 [==============================] - 9s 2ms/step - loss: 0.6874 - accuracy: 0.8195
Epoch 7/10
3912/3912 [==============================] - 9s 2ms/step - loss: 0.6553 - accuracy: 0.8270
Epoch 8/10
3912/3912 [==============================] - 10s 3ms/step - loss: 0.6351 - accuracy: 0.8294
Epoch 9/10
3912/3912 [==============================] - 10s 3ms/step - loss: 0.6143 - accuracy: 0.8348
Epoch 10/10
3912/3912 [==============================] - 9s 2ms/step - loss: 

In [152]:
def make_subject_index(subject):
    subject = okt.nouns(subject)
    subject_list = []
    for i in subject:
        subject_list.append(word_to_index.get(i,0))
    return pad_sequences(np.array([subject_list]),21)

In [222]:
subject = ""

In [223]:
x = make_subject_index(subject)

In [224]:
result = model.predict(x)
result = np.argmax(result)

1/1 [==============================] - 0s 32ms/step


In [225]:
index_to_subject.get(result)

'일본어'

In [230]:
sentence = "내가그린기린그림은잘그린기린그림이고"

In [231]:
okt.pos(sentence)

[('내', 'Noun'),
 ('가', 'Josa'),
 ('그린', 'Noun'),
 ('기린', 'Noun'),
 ('그림', 'Noun'),
 ('은', 'Josa'),
 ('잘', 'Verb'),
 ('그린', 'Noun'),
 ('기린', 'Noun'),
 ('그림', 'Noun'),
 ('이고', 'Josa')]

In [232]:
kkma.pos(sentence)

[('내가', 'NNG'),
 ('그린', 'NNG'),
 ('기린', 'NNG'),
 ('그림', 'NNG'),
 ('은', 'JX'),
 ('잘', 'MAG'),
 ('그리', 'VV'),
 ('ㄴ', 'ETD'),
 ('기린', 'NNG'),
 ('그림', 'NNG'),
 ('이', 'VCP'),
 ('고', 'ECE')]